In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
from math import sqrt

In [3]:
airbnb=pd.read_csv("AB_NYC_2019.csv")

#Remove unsignificant data
airbnb.drop(['name','id','host_name','last_review'], axis=1, inplace=True)
airbnb.fillna({'reviews_per_month':0}, inplace=True)
airbnb.head(3)

,host_id,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,2787,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,0.21,6,365
1,2845,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,0.38,2,355
2,4632,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,0.00,1,365


In [11]:
airbnb_group = airbnb.groupby('neighbourhood_group')
airbnb_group['price'].agg(['mean'])

,mean
neighbourhood_group,
Bronx,87.496792
Brooklyn,124.383207
Manhattan,196.875814
Queens,99.517649
Staten Island,114.812332


In [25]:
new_df = airbnb_group.agg({'price': ['count', 'mean']})
new_df.columns = ['Count', 'Mean']
new_df.reset_index()
new_df['Sqrt_Count'] = new_df['Count'].apply(sqrt)
new_df['Sqrt_Mean'] = new_df['Mean'].apply(sqrt)
count_median = new_df['Sqrt_Count'].median()
mean_median = new_df['Sqrt_Mean'].median()

new_df['Sqrt_Count_Div_Median'] = new_df['Sqrt_Count'] / count_median
new_df['Sqrt_Mean_Div_Median'] = new_df['Sqrt_Mean'] / mean_median
new_df[['Count', 'Mean', 'Sqrt_Count_Div_Median', 'Sqrt_Mean_Div_Median']].to_csv('district_scales_data.csv', index=False)

In [27]:
new_df2 = airbnb_group2 = airbnb.groupby(['neighbourhood_group', 'room_type']).agg({'price': ['count', 'mean']})
new_df2.columns = ['Count', 'Mean']
new_df2.reset_index()
new_df2['Sqrt_Count'] = new_df2['Count'].apply(sqrt)
new_df2['Sqrt_Mean'] = new_df2['Mean'].apply(sqrt)
new_df2

[379,
 652,
 60,
 9559,
 10132,
 413,
 13199,
 7982,
 480,
 2096,
 3372,
 198,
 176,
 188,
 9]

In [28]:
new_df2

Count        Mean  Sqrt_Count  Sqrt_Mean
neighbourhood_group room_type                                                
Bronx               Entire home/apt    379  127.506596   19.467922  11.291882
                    Private room       652   66.788344   25.534291   8.172414
                    Shared room         60   59.800000    7.745967   7.733046
Brooklyn            Entire home/apt   9559  178.327545   97.770139  13.353934
                    Private room     10132   76.500099  100.657836   8.746433
                    Shared room        413   50.527845   20.322401   7.108294
Manhattan           Entire home/apt  13199  249.239109  114.886901  15.787308
                    Private room      7982  116.776622   89.342039  10.806323
                    Shared room        480   88.977083   21.908902   9.432766
Queens              Entire home/apt   2096  147.050573   45.782093  12.126441
                    Private room      3372   71.762456   58.068925   8.471272
                    Shared room        198   69.020202   14.071247   8.307840
Staten Island       Entire home/apt    176  173.846591   13.266499  13.185090
                    Private room       188   62.292553   13.711309   7.892563
                    Shared room          9   57.444444    3.000000   7.579211